# PettingZoo, UnityEnviroment and python-neat
<p> This is a trainer for autonomous drone enviroment </p>

In [33]:
from mlagents_envs.environment import UnityEnvironment  # Import Unity environment
from mlagents_envs.envs.unity_aec_env import UnityAECEnv
from mlagents_envs.envs.unity_parallel_env import UnityParallelEnv
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from comunication_channel import AgentLogChannel
import neat

In [34]:
engine_config_channel = EngineConfigurationChannel()
engine_config_channel.set_configuration_parameters(time_scale=1)
agent_count_channel = AgentLogChannel()
env_file_name = "..\\Builds\\single-agent-env\\autonomous-drone.exe"
env = UnityEnvironment(file_name=env_file_name, worker_id=0, no_graphics=True, side_channels=[engine_config_channel, agent_count_channel])

env = UnityParallelEnv(env)
num_agents = len(env.possible_agents)
num_agents

1

In [35]:
obs = env.reset()

In [19]:
list(obs.values())[0]


array([ 6.2357119e-10,  3.0008994e-02,  1.1849005e-09,  3.0008994e-02,
        1.1967495e-07,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00,
        0.0000000e+00, -9.7397842e-02,  0.0000000e+00,  0.0000000e+00,
       -1.9296518e-01,  0.0000000e+00], dtype=float32)

In [20]:
def send_num_agents(num: int):
    agent_count_channel.send_int(data=num)   

In [21]:
# Test sending data
targetAgentsCount = 2
send_num_agents(targetAgentsCount)

Sending: 2


In [22]:
obs = env.reset()
assert len(obs) == targetAgentsCount, "Dictionary does not have exactly 2 items"

In [23]:
num_actions = env.action_space(env.possible_agents[0]).shape[0]
print(f"There is total of {num_actions} actions in enviroment")

There is total of 4 actions in enviroment


In [24]:
num_inputs = env.observation_space(env.possible_agents[0]).shape[0]
print(f"There is total of {num_inputs} inputs in enviroment")

There is total of 14 inputs in enviroment


### Create policies here

In [25]:
def eval_genome(genome, config):
    print("Evaluating genome")
    # net = neat.nn.FeedForwardNetwork.create(genome, config)
    # while env.agents:
    #     actions = {agent: env.action_space(agent).sample() for agent in env.agents}
    #     observations, rewards, terminations, infos = env.step(actions)
    # print(rewards)
    # env.reset()

In [11]:
def eval_genomes(genomes, config):
    print("Evaluating genomes")
    for _, genome in genomes:
        genome.fitness = eval_genome(genome, config)

In [12]:
def run():
    print("Came into run")

    config_path = "config"
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

    pop = neat.Population(config)
    stats = neat.StatisticsReporter()
    pop.add_reporter(stats)
    print("Starting")

    pe = neat.ParallelEvaluator(6, eval_genomes)
    winner = pop.run(pe.evaluate, 10)

In [26]:
target_agents = 1
send_num_agents(target_agents)

Sending: 1


In [28]:
env.reset()
agents = env.reset()
print(f"Num agents after reset: {len(agents)}")

Num agents after reset: 1


In [36]:
import random
import time

for i in range(3):
    target_agents = random.randint(1, 50)
    send_num_agents(target_agents)
    env.reset()
    agents = env.reset()
    print(f"Num agents after reset: {len(agents)}, and env has: {len(env.agents)}")
    while env.agents:
        actions = {agent: env.action_space(agent).sample()  for agent in env.agents} # env.action_space(agent).sample()
        observations, rewards, terminations, infos = env.step(actions)
    print(f"Num rewards after: {rewards}")


Sending: 37
Num agents after reset: 37, and env has: 37
Num rewards after: {'RotorControl?team=0?agent_id=36': -0.96860385}
Sending: 46
Num agents after reset: 46, and env has: 46
Num rewards after: {'RotorControl?team=0?agent_id=18': -0.99782336}
Sending: 28
Num agents after reset: 28, and env has: 28
Num rewards after: {'RotorControl?team=0?agent_id=23': -22.101067}


In [32]:
env.close()